<a href="https://colab.research.google.com/github/asdjgfr/my-colab/blob/master/DAIN_AI%E8%87%AA%E5%8A%A8%E8%A1%A5%E5%B8%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAIN Colab 说明


fork 来自:
https://github.com/styler00dollar/DAIN


说明：
- 复制.ipynb文件到你的网盘。
- 在谷歌云盘中新建名为"DAIN"的文件夹，或上下保持一致。
- 在下一个单元格中更改配置。
- 一张一张地运行单元格。

需要改进的内容：
- Alpha通道将被自动删除，并且不会重新添加。 与alpha相关的所有内容都会转换为黑色。
- 添加速度选择。
- 检测场景以避免插值场景变化。
- 自动恢复。
- 将“ start_frame”-“end_frame”音频从原始输入复制到最终输出。

# 配置

In [ ]:
################# 配置 ############################

# INPUT_FILEPATH：输入文件的路径（相对于Google云盘的根目录）。
# 例如，如果您将"example.mkv"文件保存在Google云盘中的"videos"文件夹中，则路径为：
# videos/example.mkv
INPUT_FILEPATH = "DAIN/input.mp4"

# OUTPUT_FILE_PATH: 输出文件的路径（相对于Google云盘的根目录）。
# 目标文件类型建议使用 mp4
OUTPUT_FILE_PATH = "DAIN/output.mp4"

################# DAIN 配置 ############################

# TARGET_FPS：目标FPS
TARGET_FPS = 90

# 输入帧
# 如果在Google云盘上有现成帧（00001.png, 00002.png）则改为云盘路径
# Google云盘地址 `/content/gdrive/My Drive/`
FRAME_INPUT_DIR = '/content/DAIN/input_frames'

# 输出帧
# 如果要将生成的帧存储到Google云盘，请使用GDrive中的位置。
# Google云盘地址 `/content/gdrive/My Drive/`
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames'

# 无缝循环
# 通过将第一个帧也用作最后一个帧来创建无缝循环。
SEAMLESS = False

# 调整大小
# 与原始输入帧相比，DAIN 帧有些“偏移/较小”。 调整大小可以部分缓解这种情况
# 将帧设置为+2px的分辨率，并以起点（1,1）将结果裁剪为原始分辨率。
# 如果没有此修复程序，DAIN往往会产生震动模糊，并且在诸如文本之类的静态元素中非常明显。
# 用户可以更改插值方法。 建议使用方法cv2.INTER_CUBIC和cv2.INTER_LANCZOS4。
# 当前默认值为 cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True

# ffmpeg生成视频后自动删除输出的PNG文件夹
AUTO_REMOVE = False

# 挂载谷歌云盘

In [ ]:
# 挂载Google云盘
# 点击下方出现的链接获取授权码，然后填入输入框回车挂载
# 挂载成功后在左侧目录中可以看到名为gdrive的文件夹
from google.colab import drive
drive.mount('/content/gdrive')
print('谷歌云盘挂载成功！')

# 检查GPU

In [ ]:
# 检查当前GPU
# 如果幸运的话，可以获得16GB的VRAM。VRAM很重要。VRAM越多，最大分辨率越高。

# 16GB：可以处理720p。1080p将导致内存不足错误。
# 8GB: 可以处理480p。720p将导致内存不足错误。

# P100: 16GB （正常）
# T4: 16GB （CUDA会失败）
# P4: 8GB （正常）
# K80: 8GB （未随机到这个GPU）

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

# 安装依赖

下一步可能需要15-20分钟。在启动时仅运行一次即可。


In [ ]:
from IPython.display import clear_output
!git clone https://github.com/asdjgfr/Colab-DAIN.git /content/DAIN

# 这步可能需要15分钟
# 构建 DAIN.
%cd /content/DAIN/my_package/
!./build.sh
print("构建第一步成功！")

# 大概需要5分钟
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("构建第二步成功！")

# 下载 pre-trained 模型
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
# 如果上面失效可以使用备用地址：
# !wget -O model_weights/best.pth https://www.2077tech.com/files/dain/best.pth

!CUDA_VISIBLE_DEVICES=0

!sudo apt-get install imagemagick imagemagick-doc

# 检测FPS

In [ ]:
# 检测输入文件的FPS。
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

if(fps/TARGET_FPS>0.5):
  print("请定义一个更高的FPS，因为没有足够的时间用于新帧。旧FPS/新FPS应低于0.5，如果尝试补帧有可能会失败。")

# 提取帧数据

In [ ]:
# ffmpeg 提取 - 从源文件生成单个帧的PNG文件。
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

%shell ffmpeg -i '/content/DAIN/{filename}' '{FRAME_INPUT_DIR}/%05d.png'

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)
  
print(f"{pngs_generated_count} 帧 PNG 生成！")

# 检查图片是否拥有透明通道
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} 命中")

# 如果拥有透明通道则删除
if "a" in channels:
  print("检测到透明通道，即将删除！")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

# 开始补帧

In [ ]:
# 补帧
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

# 复制补帧后的文件夹到谷歌云盘（可选）

In [ ]:
%shell rm -rf /content/gdrive/My\ Drive/DAIN/output_frames
%shell yes | cp -R -f /content/DAIN/output_frames /content/gdrive/My\ Drive/DAIN/output_frames

# 复制备份文件到补帧文件夹（可选）

In [ ]:
%shell yes | cp -R -f /content/gdrive/My\ Drive/DAIN/output_frames /content/DAIN/output_frames

# 优化匹配原始图像

In [ ]:
# 查找DAIN帧，放大和裁剪以匹配原始图像
import numpy as np
%cd {FRAME_OUTPUT_DIR}
if(RESIZE_HOTFIX==True):
  images = []
  for filename in os.listdir(f'{FRAME_OUTPUT_DIR}'):
    img = cv2.imread(os.path.join(f'{FRAME_OUTPUT_DIR}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False):
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      cv2.imwrite(part_filename[0]+".png", crop)

%cd /content/DAIN

# 无音频创建视频

In [ ]:
# 创建视频
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'
if(AUTO_REMOVE==True):
  !rm -rf {FRAME_OUTPUT_DIR}/*

# 有音频创建视频

In [ ]:
# 如果视频有声音则执行此单元格
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/gdrive/My Drive/{INPUT_FILEPATH}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'
if(AUTO_REMOVE==True):
  !rm -rf {FRAME_OUTPUT_DIR}/*
  !rm -rf output-audio.aac

# 删除输出帧文件夹

In [ ]:
# 删除输出PNG文件夹（训练新视频之前执行）
!rm -rf {FRAME_OUTPUT_DIR}/*